In [ ]:
# scikit learn install
!pip install scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np

In [ ]:
iris = load_iris()
x = iris.data
y = iris.target

# 붓꽃의 꽃받침과 꽃잎의 [길이, 너비]에 대한 값 -> 붓꽃의 종류 setosa / versicolor / virginica
print(x.shape, y.shape)

In [ ]:
def norm_maxmin(data, out_min=0, out_max=255):
    
    gradient = (out_max - out_min) / (np.max(data) - np.min(data))
    bias = out_min - np.min(data) * gradient
    out = data * gradient + bias

    return out

In [ ]:
for it_col in range(x.shape[1]):
    x[:, it_col] = norm_maxmin(x[:, it_col], 0, 255)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train = x_train.reshape(-1, 15, 4)
y_train = y_train.reshape(-1, 15)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [ ]:
class FCN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FCN, self).__init__()
        self.fc = nn.Linear(input_dim, 16)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(16, 8)
        self.relu2 = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(8, num_classes)
        self.relu3 = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.relu(self.fc(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))        
        return x

In [ ]:
model = FCN(input_dim=x_train.shape[2], num_classes=3)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 10000
for e in range(epochs):
    for b in range(len(x_train)):
        outputs = model(x_train[b])
        loss = criterion(outputs, y_train[b])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (e+1) % 1000 == 0:
        print(f'Epoch {e + 1} / {epochs}, Loss : {loss.item()}')
        with torch.no_grad():

            model.eval()

            correct = 0
            total = 0
            outputs = model(x_test)
            _, predicted = torch.max(outputs.data, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
            print('Accuracy of the model on test data: {} %'.format(100 * correct / total))
        model.train()

In [ ]:
# 테스트
with torch.no_grad():

    model.eval()

    correct = 0
    total = 0
    outputs = model(x_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()
    print('Accuracy of the model on test data: {} %'.format(100 * correct / total))
